線形回帰であるリッジ回帰、LASSO、Elastic Netを実行してみる。

In [1]:
import pandas as pd
import sys

from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge


print(sys.version_info)

sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)


### 1. データの準備

Toyデータを用意する。  
[make_regression](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html)関数を使うと自身が望むような回帰分析の検証のためのサンプルデータを用意することができる。  
今回は`サンプル数=100`, `説明変数の数=10`, そして再現性を担保するために`random_state=0`, `shuffle=False`とした。

In [2]:
X, y = make_regression(n_samples=1000, n_features=10,
                                                random_state=0, shuffle=False)
X = pd.DataFrame(X)

In [3]:
X.head()

,0,1,2,3,4,5,6,7,8,9
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599
1,0.144044,1.454274,0.761038,0.121675,0.443863,0.333674,1.494079,-0.205158,0.313068,-0.854096
2,-2.552990,0.653619,0.864436,-0.742165,2.269755,-1.454366,0.045759,-0.187184,1.532779,1.469359
3,0.154947,0.378163,-0.887786,-1.980796,-0.347912,0.156349,1.230291,1.202380,-0.387327,-0.302303
4,-1.048553,-1.420018,-1.706270,1.950775,-0.509652,-0.438074,-1.252795,0.777490,-1.613898,-0.212740


In [4]:
y[:5]

array([ 300.2064792 ,  194.27686437,   13.25754564,  -24.6027897 ,
        -98.76061926])

サンプルサイズと説明変数の数は`shape`に保存されている。

In [5]:
X.shape

(1000, 10)

### 2. リッジ回帰

#### 2.1 機械学習

Ridgeクラスのコンストラクタの引数`alpha`が講義資料の`λ`に相当する。今回は`alpha=1`として学習を行ってみる。

In [6]:
regr = Ridge(alpha=1.0)
regr.fit(X, y) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

#### 2.2 説明変数の重み、係数

説明変数の重みは`coef_`、係数は`intercept_`に保存されている。[参考ドキュメント](http://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

In [7]:
regr.coef_

array([  9.56477279,  40.1465737 ,  52.49713485,  73.14621686,
        13.51119737,  43.97620616,  69.47748865,  63.21789581,
         8.40281783,  33.60209855])

In [8]:
regr.intercept_

-0.010409753984825798

#### 2.3 予測および性能評価

予測値を求めるには`predict`メソッドを用いる。

In [9]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 299.84797707,  194.04108457,   13.21386842,  -24.58870742,
        -98.68132918])

また`score`メソッドを用いることで実測値と予測値の間の相関係数の2乗(いわゆるR<sup>2</sup>)を求めることができる。

In [10]:
regr.score(X, y)

0.99999866516220048

#### 2.4 ペナルティ項の重み(引数alpha, 講義資料ではλ)を変えてみる

`alpha=0.5`とし説明変数の重みや相関係数の2乗が変化するか確認してみる。

In [11]:
regr = Ridge(alpha=0.5)
regr.fit(X, y) 

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [12]:
regr.coef_

array([  9.56572052,  40.16931242,  52.5260632 ,  73.18969112,
        13.51953007,  43.9996237 ,  69.52131158,  63.25331236,
         8.40325475,  33.62287967])

In [13]:
regr.intercept_

-0.0052077530301595232

In [14]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 300.02712346,  194.15890573,   13.23569097,  -24.59574554,
        -98.72095804])

In [15]:
regr.score(X, y)

0.99999966589939004

わずかであるが説明変数の重みが変化したり、相関係数の2乗が大きくなったりした。